In [46]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [47]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [48]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [49]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [50]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [51]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [52]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [53]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [54]:
# Use 70% of the data for training and the remaineder for testing
split = int(.7 * len(X))
X_train = X[:split - 1]
X_test = X[split:]
y_train = y[:split -1]
y_test = y[split:]


In [55]:
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

#Fit
x_scaler.fit(X_train)
y_scaler.fit(y_train)

#Scale the training data
X_train = x_scaler.transform(X_train)
y_train = y_scaler.transform(y_train)

#Scale the y test
x_test = x_scaler.transform(X_test)
y_test = y_scaler.transform(y_test)

In [56]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [58]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

model = Sequential()
model.add(LSTM(
    units=30, return_sequences=True,
    input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=30))
model.add(Dropout(0.2))
model.add(Dense(1))

In [78]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [79]:
# Summarize the model
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 10, 30)            3840      
                                                                 
 dropout_8 (Dropout)         (None, 10, 30)            0         
                                                                 
 lstm_9 (LSTM)               (None, 10, 30)            7320      
                                                                 
 dropout_9 (Dropout)         (None, 10, 30)            0         
                                                                 
 lstm_10 (LSTM)              (None, 30)                7320      
                                                                 
 dropout_10 (Dropout)        (None, 30)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                

In [80]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
batch_size = 1000
epochs = 10
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
)

Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 0.2148
Epoch 2/10
1/1 [==============================] - 0s 42ms/step - loss: 0.1957
Epoch 3/10
1/1 [==============================] - 0s 40ms/step - loss: 0.1787
Epoch 4/10
1/1 [==============================] - 0s 38ms/step - loss: 0.1614
Epoch 5/10
1/1 [==============================] - 0s 43ms/step - loss: 0.1432
Epoch 6/10
1/1 [==============================] - 0s 37ms/step - loss: 0.1276
Epoch 7/10
1/1 [==============================] - 0s 37ms/step - loss: 0.1112
Epoch 8/10
1/1 [==============================] - 0s 48ms/step - loss: 0.0915
Epoch 9/10
1/1 [==============================] - 0s 37ms/step - loss: 0.0763
Epoch 10/10
1/1 [==============================] - 0s 42ms/step - loss: 0.0599


In [88]:
# Evaluate the model
model.evaluate(X_test, y_test)

5/5 [==============================] - 1s 4ms/step - loss: 0.1138


0.11380095779895782

In [97]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

NameError: name 'scaler' is not defined

In [98]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

NameError: name 'real_prices' is not defined

In [86]:
# Plot the real vs predicted values as a line chart
# YOUR CODE HERE!